In [1]:
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout 
from keras.preprocessing.image import ImageDataGenerator


In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [3]:


# @hidden_cell
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='HuoVQ5ruKaK_Vdj4rN7FV4G_qmPUBMMoEQwLHMh4GbAo',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'aipowerednutritionanalyzerforfitn-donotdelete-pr-70uw3yez5qbpw2'
object_key = 'Dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']



In [4]:
from io import BytesIO
import zipfile 

unzip =zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)


In [5]:
pwd

'/home/wsuser/work'

In [6]:
x_train = train_datagen.flow_from_directory(
    r'Dataset/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test_datagen.flow_from_directory(
    r'Dataset/TEST_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 4118 images belonging to 5 classes.
Found 929 images belonging to 5 classes.


In [7]:
print(x_train.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [8]:
print(x_test.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [9]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 995, 1: 1354, 2: 1019, 3: 275, 4: 475})

In [10]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) 



In [11]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [12]:

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [26]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

/tmp/wsuser/ipykernel_164/549542485.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
824/824 [==============================] - 45s 54ms/step - loss: 0.6044 - accuracy: 0.7613 - val_loss: 0.5241 - val_accuracy: 0.7944
Epoch 2/20
824/824 [==============================] - 45s 55ms/step - loss: 0.4107 - accuracy: 0.8448 - val_loss: 0.5083 - val_accuracy: 0.7901
Epoch 3/20
824/824 [==============================] - 44s 54ms/step - loss: 0.3856 - accuracy: 0.8587 - val_loss: 0.4260 - val_accuracy: 0.8332
Epoch 4/20
824/824 [==============================] - 44s 53ms/step - loss: 0.3569 - accuracy: 0.8657 - val_loss: 0.4178 - val_accuracy: 0.8471
Epoch 5/20
824/824 [==============================] - 44s 53ms/step - loss: 0.3306 - accuracy: 0.8747 - val_loss: 0.4230 - val_accuracy: 0.8428
Epoch 6/20
824/824 [==============================] - 45s 54ms/step - loss: 0.3061 - accuracy: 0.8844 - val_loss: 0.4211 - val_accuracy: 0.8299
Epoch 7/20
824/824 [==============================] - 45s 54ms/step - loss: 0.2897 - accuracy: 0.8885 - val_loss: 0.3864 - val_accuracy:

In [13]:

classifier.save('nutrition.h5')

In [27]:
!tar -zcvf image-classification-model_new.tgz nutrition.h5

nutrition.h5


In [28]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("nutrition.h5")
from tensorflow.keras.preprocessing import image

In [29]:
img =image.load_img(r"Dataset/TRAIN_SET/ORANGE/n07749192_1137.jpg",grayscale=False, target_size= (64,64))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0) 
y =np.argmax(model.predict(x),axis=1)
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
index[y[0]]

'PINEAPPLE'

In [30]:
!pip install ibm_watson_machine_learning

In [31]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

     |████████████████████████████████| 312 kB 23.7 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 20.0 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.
     |████████████████████████████████| 454.4 MB 34 kB/s s eta 0:00:01██                    | 171.8 MB 84.9 MB/s eta 0:00:04
     |████████████████████████████████| 14.9 MB 31.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 76.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 82.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 73.7 MB/s eta 0:00:01
     |████████████████████████████████| 462 k

In [33]:
from ibm_watson_machine_learning import APIClient

In [34]:
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"mi06K7pucJqDfgSQmtjRHfX3LUfx5Nz8kkdIVnSIuRHm"
}

In [35]:
client=APIClient(wml_credentials)

In [36]:
client

In [37]:
client.spaces.list()


Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------------  ------------------------
ID                                    NAME            CREATED
5f79be14-2093-4939-a654-fa05968ddba8  Classification  2022-11-18T12:41:02.378Z
------------------------------------  --------------  ------------------------


In [41]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [42]:
space_uid = guid_from_space_name(client,'Classification')
print("Space UID =" + space_uid)

Space UID =5f79be14-2093-4939-a654-fa05968ddba8


In [47]:
client.set.default_space(space_uid)

'SUCCESS'

In [48]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [49]:
software_space_id = client.software_specifications.get_id_by_name("tensorflow_rt22.1-py3.9")


In [50]:
software_space_id


'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [51]:
!tar -zcvf nutrition-classification-model.tgz 'nutrition.h5'


nutrition.h5


In [52]:
model_details=client.repository.store_model(model ='nutrition-classification-model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN Model Building",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_space_id
})

In [53]:
model_id = client.repository.get_model_id(model_details)

In [54]:
model_id


'66453365-9d3b-49b2-a37e-7a19017aa7ed'

In [55]:
client.repository.download(model_id,'nutrition.tgz')

Successfully saved model content to file: 'nutrition.tgz'


'/home/wsuser/work/nutrition.tgz'